In [1]:
! pip install python-dotenv
! pip install -qU langchain-openai
! pip install -qU langchain_community beautifulsoup4
! pip install faiss-cpu
! pip install -qU langchain-groq
! pip install langchain_core

In [2]:
env_content = """

GROQ_API_KEY=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
"""

with open(".env", "w") as f:
    f.write(env_content.strip())

print(".env file created successfully!")


.env file created successfully!


In [3]:
import os
from dotenv import load_dotenv
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
load_dotenv()


True

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f6266010790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f626607ae50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Jarif and I am a AI Engineer")])

AIMessage(content="Nice to meet you, Jarif. As an AI Engineer, I'm sure you're working on some exciting projects, especially with the rapid advancements in AI technology. What areas of AI are you currently focusing on, such as Natural Language Processing (NLP), computer vision, or perhaps reinforcement learning?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 48, 'total_tokens': 109, 'completion_time': 0.081333333, 'prompt_time': 0.003957791, 'queue_time': 0.023249107, 'total_time': 0.085291124}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-63529cdf-058e-4773-8e19-7e5123803f2a-0', usage_metadata={'input_tokens': 48, 'output_tokens': 61, 'total_tokens': 109})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Jarif and I am a Chief AI Engineer"),
        AIMessage(content="Hello Jarif! It's nice to meet you. \n\nAs a  AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Jarif, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 119, 'total_tokens': 134, 'completion_time': 0.02, 'prompt_time': 0.008525592, 'queue_time': 0.018840036999999997, 'total_time': 0.028525592}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-73b939dd-e5e1-456e-86c9-fc501dc95261-0', usage_metadata={'input_tokens': 119, 'output_tokens': 15, 'total_tokens': 134})

## Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Jarif and I am a  AI Engineer")],
    config=config
)

In [9]:
response.content

"Nice to meet you, Jarif. Congratulations on pursuing a career in AI engineering, it's a fascinating field with a lot of growth potential. What specific areas of AI engineering are you interested in or have experience with?"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Jarif, and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 108, 'total_tokens': 122, 'completion_time': 0.018666667, 'prompt_time': 0.007913043, 'queue_time': 0.019241577000000003, 'total_time': 0.02657971}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-17ffc1c9-4215-4c87-b36c-cee3c8b2b674-0', usage_metadata={'input_tokens': 108, 'output_tokens': 14, 'total_tokens': 122})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name. I'm a text-based AI assistant, and our conversation just started, so I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you. How are you doing today? Is there something I can help you with or would you like to chat about something in particular?"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

## Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model
chain.invoke({"messages":[HumanMessage(content="Hi My name is Protik")]})

with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [15]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Protik")],
    config=config
)

response

AIMessage(content="Hello Protik, it's nice to meet you. I'm here to help with any questions or information you may need. Is there something specific I can assist you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 58, 'total_tokens': 100, 'completion_time': 0.056, 'prompt_time': 0.005628218, 'queue_time': 0.023642081, 'total_time': 0.061628218}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-b78fe81f-12fe-463a-8ca2-226b4411a0d6-0', usage_metadata={'input_tokens': 58, 'output_tokens': 42, 'total_tokens': 100})

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Protik.'

## Add more complexity

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Jarif")],"language":"Bangla"})
response.content

'আপনার নাম জারিফ হলো। আমি আপনার সহায়তা করার জন্য তৈরি। আপনার কোনো প্রশ্ন আছে?'

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
onfig = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am jarif")],"language":"Bangla"},
    config=config
)
repsonse.content

'Jarif হলো আপনার নাম। আমার সাথে কথা বলতে আপনার কোন সমস্যা আছে?'

In [20]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "bengali"},
    config=config,
)
response.content

'আপনার নাম জারিফ।'

### Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. 'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages



In [21]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [22]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I don't have that information. You haven't told me what your favorite ice cream flavor is. Would you like to share?"

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't mention your name earlier. I'm happy to chat with you, but I don't have any information about your identity. Would you like to share your name with me?"

In [24]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a math problem yet. This is the beginning of our conversation. What would you like to ask or discuss?"